In [1]:
# Imports needed

import os
import PIL
import pathlib

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
tf.config.list_physical_devices()

Hello world


In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH  = 224


# Data path and directories
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'Dataset/train_validation'))
data_dir = pathlib.Path(data_dir)

data_count = []

for animal in data_dir.glob('*'):
    name = str(animal).split("\\")[-1]
    number = len(list(data_dir.glob(f'*{name}/*.jpg')))
    data_count.append((name, number))

print(data_count)

In [ ]:
# Build the class names

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*')])
CLASS_NAMES

In [ ]:
# Create Training and Validation dataset

# Create the image_generator
image_generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2,
)

# Create the Training dataset generator
train_data_gen = image_generator.flow_from_directory(
    directory=data_dir,
    batch_size=BATCH_SIZE,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    subset='training',
)

# Crate the validation dataset generator
validation_data_gen = image_generator.flow_from_directory(
    directory=data_dir,
    batch_size=BATCH_SIZE,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    subset='validation',
)

In [ ]:
# Here, let's define a function, show_batch() for inspecting a batch

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5, 5, n+1)
        plt.imshow(image_batch[n])
        plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
        plt.axis('off')

        
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

In [1]:
# Import VGG16

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [ ]:
n_classes = len(CLASS_NAMES)
input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
n_epochs = 10
n_steps = train_data_gen.samples // BATCH_SIZE
n_val_steps = validation_data_gen.samples // BATCH_SIZE

In [ ]:
# Build the model

model_base = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
)

# make the rest of the layers untrainable
for layer in model_base.layers:
    layer.trainable = False

# Add layers on top of the model a process called
# Fine tunning
top_model = model_base.output
top_model = tf.keras.layers.Flatten(name="flatten")(top_model)
top_model = tf.keras.layers.Dense(1024, activation='relu')(top_model)
top_model = tf.keras.layers.Dropout(0.2)(top_model)
output_layer = tf.keras.layers.Dense(n_classes, activation='softmax')(top_model)

model = Model(inputs=model_base.input, outputs=output_layer)

model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# We would like to save the best model whilt training

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(
    filepath='tl_model_v1.weights.best.hdf5',
    save_best_only=True,
    verbose=1
)

#Early stopping to avoid overfitting of model
early_stop = EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=5
)


In [ ]:
with tf.device('/GPU:0'):
    history = model.fit(
        train_data_gen,
        epochs=n_epochs,
        validation_data=validation_data_gen,
        steps_per_epoch=n_steps,
        validation_steps=n_val_steps,
        callbacks=[
            tl_checkpoint_1
        ],
    )

In [ ]:
# Visualize the training and validation accuracies

plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
# Visualize the training and validation losses

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
model.save("camera_trap_model.hdf5")

In [ ]:
image_batch, _ = next(validation_data_gen)

prediction = model.predict(image_batch)

In [ ]:
PREDICTED_CLASS_NAMES = []

for i in range(len(prediction)):
    score = prediction[i]
    name = CLASS_NAMES[score.argmax()]
    PREDICTED_CLASS_NAMES.append(name)

In [ ]:
def show_predicted_batch(image_batch, labels):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5, 5, n+1)
        plt.imshow(image_batch[n])
        plt.title(labels[n])
        plt.axis('off')
        
show_predicted_batch(image_batch, PREDICTED_CLASS_NAMES)